# Exam 2:

## Question 1: Emotion Detection in Text

In [2]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.6 MB/s eta 0:00:00


In [26]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the emotions
model = BayesianNetwork([('Emotion', 'happy'), ('Emotion', 'joyful'), ('Emotion', 'great'),
                         ('Emotion', 'sad'), ('Emotion', 'down'), ('Emotion', 'angry'),
                         ('Emotion', 'mad'), ('Emotion', 'frustrated')])

# Define Conditional Probability Distributions
emotion = TabularCPD(variable='Emotion', variable_card=3,
                     values=[[1/3], [1/3], [1/3]],
                     state_names={'Emotion': ['happy', 'sad', 'angry']})

happy = TabularCPD(variable='happy', variable_card=2,
                       values=[[0.75, 0.95, 0.90],
                               [0.25, 0.05, 0.10]],
                       evidence=['Emotion'],
                       evidence_card=[3],
                       state_names={'happy': ['No', 'Yes'],
                                    'Emotion': ['happy', 'sad', 'angry']})

joyful = TabularCPD(variable='joyful', variable_card=2,
                        values=[[0.875, 0.98, 0.97],
                                [0.125, 0.02, 0.03]],
                        evidence=['Emotion'],
                        evidence_card=[3],
                        state_names={'joyful': ['No', 'Yes'],
                                     'Emotion': ['happy', 'sad', 'angry']})

great = TabularCPD(variable='great', variable_card=2,
                      values=[[0.875, 0.97, 0.98],
                              [0.125, 0.03, 0.02]],
                      evidence=['Emotion'],
                      evidence_card=[3],
                      state_names={'great': ['No', 'Yes'],
                                   'Emotion': ['happy', 'sad', 'angry']})

sad = TabularCPD(variable='sad', variable_card=2,
                     values=[[0.95, 0.80, 0.95],
                             [0.05, 0.20, 0.05]],
                     evidence=['Emotion'],
                     evidence_card=[3],
                     state_names={'sad': ['No', 'Yes'],
                                  'Emotion': ['happy', 'sad', 'angry']})

down = TabularCPD(variable='down', variable_card=2,
                      values=[[0.97, 0.875, 0.98],
                              [0.03, 0.125, 0.02]],
                      evidence=['Emotion'],
                      evidence_card=[3],
                      state_names={'down': ['No', 'Yes'],
                                   'Emotion': ['happy', 'sad', 'angry']})

angry = TabularCPD(variable='angry', variable_card=2,
                       values=[[0.98, 0.97, 0.85],
                               [0.02, 0.03, 0.15]],
                       evidence=['Emotion'],
                       evidence_card=[3],
                       state_names={'angry': ['No', 'Yes'],
                                    'Emotion': ['happy', 'sad', 'angry']})

mad = TabularCPD(variable='mad', variable_card=2,
                     values=[[0.99, 0.95, 0.875],
                             [0.01, 0.05, 0.125]],
                     evidence=['Emotion'],
                     evidence_card=[3],
                     state_names={'mad': ['No', 'Yes'],
                                  'Emotion': ['happy', 'sad', 'angry']})

frustrated = TabularCPD(variable='frustrated', variable_card=2,
                            values=[[0.995, 0.99, 0.90],
                                    [0.005, 0.01, 0.10]],
                            evidence=['Emotion'],
                            evidence_card=[3],
                            state_names={'frustrated': ['No', 'Yes'],
                                         'Emotion': ['happy', 'sad', 'angry']})
# Add CPDs to the model
model.add_cpds(emotion, happy, joyful, great, sad, down, angry, mad, frustrated)

# Check if the model is valid
assert model.check_model()

# Create an inference object
inference = VariableElimination(model)

def classify_word(word):
    if isinstance(word, list):
        word = word[0]

    if word not in model.nodes():
        return "Error: Word not in the network"

    result = inference.query(variables=['Emotion'], evidence={word: 'Yes'})
    return result.values

def classify_multiple_words(words):
    valid_words = [word for word in words if word in model.nodes()]
    if not valid_words:
        return "Error: No valid words found in the network"

    evidence = {word: 'Yes' for word in valid_words}
    result = inference.query(variables=['Emotion'], evidence=evidence)
    return result.values

# Main
def process_text(text):
    # Convert to lowercase
    text = text.lower()

    # Split the text into words
    words = text.split()

    # Remove personal pronouns and punctuation
    personal_pronouns = set(["i", "me", "my", "mine", "i'm", "we", "us", "our", "ours",
                             "you", "your", "yours", "he", "him", "his",
                             "she", "her", "hers", "it", "its",
                             "they", "them", "their", "theirs"])
    words = [word.strip(".,!?") for word in words if word not in personal_pronouns]

    return words

# Example usage
text = input("Enter your emotional text: ")
processed_words = process_text(text)

if len(processed_words) > 1:
    classification = classify_multiple_words(processed_words)
else:
    classification = classify_word(processed_words)

if isinstance(classification[0], float):
  print(f"The result probability is:\nHappy: {classification[0]}\nSad: {classification[1]}\nAngry: {classification[2]}")
else:
  print(classification)

Enter your emotional text: g
Error: Word not in the network


## Question 2: Smart home lighting system

In [28]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.9 MB/s eta 0:00:00


In [48]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Define fuzzy variables
time_of_day = ctrl.Antecedent(np.arange(0, 24, 1), 'time_of_day')
occupancy = ctrl.Antecedent(np.arange(0, 2, 1), 'occupancy')
lighting_intensity = ctrl.Consequent(np.arange(0, 101, 1), 'lighting_intensity')
preference = ctrl.Antecedent(np.arange(1, 4, 1), 'preference')

# Define membership functions for time of day
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [6, 9, 12])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [12, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [18, 21, 24])
time_of_day['night'] = fuzz.trimf(time_of_day.universe, [0, 3, 6])

# Define membership functions for occupancy
occupancy['unoccupied'] = fuzz.trimf(occupancy.universe, [0, 0, 1])
occupancy['occupied'] = fuzz.trimf(occupancy.universe, [0, 1, 1])

# Define membership functions for lighting intensity
lighting_intensity['off'] = fuzz.trimf(lighting_intensity.universe, [0, 0, 25])
lighting_intensity['low'] = fuzz.trimf(lighting_intensity.universe, [25, 25, 50])
lighting_intensity['medium'] = fuzz.trimf(lighting_intensity.universe, [50, 50, 75])
lighting_intensity['high'] = fuzz.trimf(lighting_intensity.universe, [75, 75, 100])

# Define membership functions for user preference
preference['low'] = fuzz.trimf(preference.universe, [1, 1, 2])
preference['medium'] = fuzz.trimf(preference.universe, [1, 2, 3])
preference['high'] = fuzz.trimf(preference.universe, [2, 3, 3])

# Define fuzzy rules
rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'] & preference['high'], lighting_intensity['medium'])
rule2 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'] & preference['medium'], lighting_intensity['low'])
rule3 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'] & preference['low'], lighting_intensity['off'])

rule4 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'] & preference['high'], lighting_intensity['low'])
rule5 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'] & preference['medium'], lighting_intensity['off'])
rule6 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'] & preference['low'], lighting_intensity['off'])

rule7 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'] & preference['high'], lighting_intensity['high'])
rule8 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'] & preference['medium'], lighting_intensity['medium'])
rule9 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'] & preference['low'], lighting_intensity['low'])

rule10 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting_intensity['off'])
rule11 = ctrl.Rule(time_of_day['night'] & occupancy['occupied'] & preference['high'], lighting_intensity['medium'])
rule12 = ctrl.Rule(time_of_day['night'] & occupancy['occupied'] & preference['medium'], lighting_intensity['low'])
rule13 = ctrl.Rule(time_of_day['night'] & occupancy['occupied'] & preference['low'], lighting_intensity['off'])


rule14 = ctrl.Rule(occupancy['occupied'] | occupancy['unoccupied'] & preference['high'], lighting_intensity['off'])
rule15 = ctrl.Rule(occupancy['occupied'] | occupancy['unoccupied'] & preference['medium'], lighting_intensity['off'])
rule16 = ctrl.Rule(occupancy['occupied'] | occupancy['unoccupied'] & preference['low'], lighting_intensity['off'])

# Control system creation
lighting_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, rule11, rule12, rule13, rule14, rule15, rule16])
lighting_simulation = ctrl.ControlSystemSimulation(lighting_control)

# Inputs and computation
example_times = [7, 13, 19, 23]
example_occupancies = [1, 0, 1, 0]
preference = input("Set your preference (high: 3, medium: 2, low:1)")

# Input error managing
options = ['1', '2', '3']
while preference not in options:
  preference = input("Error: Wrong value \nSet your preference (high: 3, medium: 2, low:1)")

for time, occupancy in zip(example_times, example_occupancies):
    lighting_simulation.input['time_of_day'] = time
    lighting_simulation.input['occupancy'] = occupancy
    lighting_simulation.input['preference'] = int(preference)
    lighting_simulation.compute()
    print(f"At {time} o'clock and occupancy {occupancy}, the lighting intensity is {lighting_simulation.output['lighting_intensity']:.2f}")


Set your preference (high: 3, medium: 2, low:1)3
At 7 o'clock and occupancy 1, the lighting intensity is 27.30
At 13 o'clock and occupancy 0, the lighting intensity is 8.33
At 19 o'clock and occupancy 1, the lighting intensity is 36.46
At 23 o'clock and occupancy 0, the lighting intensity is 8.33
